In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import numpy as np
import logging
import os
import matplotlib.pyplot as plt
import pickle
from transformers import ViTFeatureExtractor, ViTModel
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from scipy.special import softmax

IMG_SHAPE = 224

import torch
import clip
print(torch.__version__)
print("torch detects GPU: " , torch.cuda.is_available())

1.12.1+cu116
torch detects GPU:  True


# Resizing the images

In [2]:
# load every image in the food folder and resize them to 224x224, then save them in a new folder

# for filename in os.listdir('food'):
#     img = Image.open('food/' + filename)
#     img = img.resize((IMG_SHAPE, IMG_SHAPE))
#     img.save('food_'+str(IMG_SHAPE)+'/' + filename)


# Extracting the features

In [3]:
image = Image.open('food_'+str(IMG_SHAPE)+'/00007.jpg')

feature_extractor = AutoFeatureExtractor.from_pretrained("nateraw/food")
model = AutoModelForImageClassification.from_pretrained("nateraw/food")

#put the models on the GPU
# feature_extractor = feature_extractor.to('cuda')
model = model.to('cuda')

#remove the classifier layer
model.classifier = torch.nn.Identity()
model.vit.layernorm = torch.nn.Identity()
# print(model)

with torch.no_grad():
    np.set_printoptions(suppress=True)
    inputs = feature_extractor(images=image, return_tensors="pt").to('cuda')
    # print(inputs)
    outputs = model(**inputs)
    # soft = outputs.logits.softmax(dim=-1).cpu().numpy()
    # print(np.argmax(soft))
    # print(outputs)
    print(outputs.logits.cpu().numpy().shape)
    print(outputs.logits.cpu().numpy())



(1, 101)
[[-0.10903698  0.7071282  -0.1231105  -0.48549244 -0.10596184 -0.34187785
   0.27113667 -0.31629828  1.0921512  -0.27079928 -1.7905712  -0.17101659
   0.13042854 -0.5239889   1.7427368  -0.19157255 -0.7800584   1.1850092
   0.13878821 -0.34238803 -0.15138647 10.107201    2.871378    0.37200615
  -0.9084653  -0.40659168 -0.7088722  -0.24617077 -0.3055647   0.9293847
  -0.31592616  1.161826   -0.4173769  -0.5094079  -0.8763412  -0.7942207
   0.07240397  0.67694217  0.5420245  -1.3170156  -0.35400978 -0.5168432
   0.17636983 -0.37139076  0.0949786  -0.8484421   0.20012502 -1.0307158
   0.12617676 -0.4830395  -0.49172133  0.20857431 -0.86916643  0.15447226
  -0.2308073  -0.45156053 -0.69761574 -0.6635288  -0.25926018 -0.17530335
  -0.7440919   0.11873879 -0.74085987  0.42552963 -0.04901424 -0.67083704
   0.4520535  -0.40815505 -0.33330125 -0.8885472   0.35710278  0.59184206
  -0.16199313 -0.32626444 -0.2956888   0.01370557 -0.38568968 -0.1664199
  -0.5671209   0.42489588 -0.380854

In [4]:
# dictionary with all the image names and their features
features = {}

# convert every image into features
line_id = 0
for filename in os.listdir('food_'+str(IMG_SHAPE)+''):
    id = int(filename.replace(".jpg", ""))
    # print(id)
    line_id += 1
    image = Image.open('food_'+str(IMG_SHAPE)+'/' + filename)
    with torch.no_grad():
        inputs = feature_extractor(images=image, return_tensors="pt").to('cuda')
        feature = model(**inputs).logits
        feature = np.array(feature.cpu().numpy(), dtype=np.float32)
        feature = feature.reshape(-1)
        features[filename.replace(".jpg","")] = feature
        # print(feature[0])
        if line_id % 100 == 0:
            print(line_id)
            # break
        if id == 5001:
            break

# save the features dictionary to a file using pickle
with open('features_vit.pickle', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [5]:

# features = {}
# with open('features_mn.pickle', 'rb') as handle:
#     features = pickle.load(handle)
    
print(features['00000'].shape)
print(features['00000'])
print(np.array(features['00000'][0], dtype=np.float32))
print(np.array(features['00001'][0], dtype=np.float32))
print(np.array(features['00002'][0], dtype=np.float32))
print(len(features))

(101,)
[-0.15641706 -0.04846567 -0.5680937  -0.30306917 -0.2759278   1.1834544
 -0.90819067  0.34527883 -0.3602629  -0.06251695 -0.26097277  0.46102375
 -0.7009203  -0.23648539 -0.2765711  -0.4013498  -0.2639648  -0.30348867
  0.32166398  0.35783684 -0.37837955 -0.03505383 -0.36808982  0.53418356
 -0.9663702  -0.18616615  0.12631494 -0.19195983  0.22505593 -0.29878506
 -0.4233447   0.03000382  0.07663471  2.429741   -0.90852755 -0.23284523
  0.07362539  1.3784795  -0.08516523 -0.46301675  1.0649537  -0.17978667
 -0.8194879   0.17889462 -0.04968277 -1.2409251   0.09764299 -0.20737815
  1.2718052  -0.4931599   1.0167762   0.17911531  1.1381528   0.22722667
  0.37812662 -0.97233623 -0.9378077  -0.37787005 -0.3437252  -0.84179485
 -1.0167674  -1.1358867   0.30492723  0.32368082  0.54670167  1.226591
 -0.07247812 -0.6632689  -0.5457843  -0.22663414  1.6898233  -0.40640193
 -0.7198109  -0.01986106 -0.44494003  0.44223353 -0.06518041 -0.299631
  0.5368905  -0.05286302  0.21921077 -0.27373835 

In [5]:
# features = {}

# with open('features_vit2.pickle', 'rb') as handle:
#     features = pickle.load(handle)
#     for key in features:
#         features[key] = softmax(features[key])
#     # print(np.array(features['00000'], dtype=np.float32))
#     # print(np.array(features['00001'][0], dtype=np.float32))
#     # print(np.array(features['00002'][0], dtype=np.float32))


# with open('features_vit3.pickle', 'wb') as handle:
#     pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)